In [1]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\carlo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Configuración Selenium
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = "https://www.turismo.gal/localizador-de-recursos?langId=es_ES"
driver.get(url)

print("✅ Selecciona manualmente los filtros que quieras (municipio, interés, etc.)")
print("⏳ Tienes 30 segundos para hacerlo antes de que empiece la extracción...")

time.sleep(30)

wait = WebDriverWait(driver, 20)
data = []
pagina = 1

while True:
    print(f"📄 Extrayendo página {pagina}...")

    # Esperar resultados
    resultados = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.clearfix.list-item"))
    )

    for r in resultados:
        try:
            nombre_elem = r.find_element(By.CSS_SELECTOR, "a.title.mod-href")
            nombre = nombre_elem.text.strip()
            link = nombre_elem.get_attribute("href")  # enlace del nombre
        except:
            nombre = None
            link = None

        try:
            direccion = r.find_element(By.CSS_SELECTOR, "div.col-sm-12.address .media-body").text.strip()
        except:
            direccion = None

        try:
            detalle_raw = r.find_element(By.CSS_SELECTOR, "small.show").text.strip()
            if "-" in detalle_raw:
                partes = detalle_raw.split("-", 1)
                tipo = partes[0].strip()
                categoria = partes[1].strip()
            else:
                tipo = detalle_raw.strip()
                categoria = None
        except:
            detalle_raw = None
            tipo = None
            categoria = None

        try:
            telefono = r.find_element(By.CSS_SELECTOR, "div.telefono").text.strip()
        except:
            telefono = None

        try:
            email_elem = r.find_element(By.CSS_SELECTOR, "a[href^='mailto:']")
            email = email_elem.text.strip()
        except:
            email = None

        data.append({
            "nombre": nombre,
            "link": link,
            "direccion": direccion,
            "tipo": tipo,
            "categoria": categoria,
            "telefono": telefono,
            "email": email
        })

    # Verificar si el botón siguiente está deshabilitado
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a.btn.btn-default.btn-pagination[title='Siguiente']")
        if "disabled" in next_button.get_attribute("class"):
            print("🚩 Última página alcanzada.")
            break
        else:
            driver.execute_script("arguments[0].click();", next_button)
            pagina += 1
            time.sleep(3)
    except:
        print("🚩 No se encontró botón siguiente. Fin.")
        break

# Guardar CSV
df = pd.DataFrame(data)
df.to_csv("ServiciosTurisiticos.csv", index=False, encoding="utf-8-sig")

print("✅ Datos guardados en ServiciosTurisiticos.csv")
driver.quit()


✅ Selecciona manualmente los filtros que quieras (municipio, interés, etc.)
⏳ Tienes 30 segundos para hacerlo antes de que empiece la extracción...
📄 Extrayendo página 1...
📄 Extrayendo página 2...
📄 Extrayendo página 3...
📄 Extrayendo página 4...
📄 Extrayendo página 5...
📄 Extrayendo página 6...
📄 Extrayendo página 7...
📄 Extrayendo página 8...
📄 Extrayendo página 9...
📄 Extrayendo página 10...
📄 Extrayendo página 11...
📄 Extrayendo página 12...
📄 Extrayendo página 13...
📄 Extrayendo página 14...
📄 Extrayendo página 15...
📄 Extrayendo página 16...
📄 Extrayendo página 17...
📄 Extrayendo página 18...
📄 Extrayendo página 19...
📄 Extrayendo página 20...
📄 Extrayendo página 21...
📄 Extrayendo página 22...
📄 Extrayendo página 23...
📄 Extrayendo página 24...
📄 Extrayendo página 25...
📄 Extrayendo página 26...
🚩 Última página alcanzada.
✅ Datos guardados en ServiciosTurisiticos.csv


In [16]:
import pandas as pd
import glob
import os

# Carpeta donde tienes los CSV
carpeta = "datos/"  # cambia si están en otra ruta

# Buscar todos los CSV en la carpeta
archivos = glob.glob(os.path.join(carpeta, "*.csv"))

print(f"🔎 Encontrados {len(archivos)} archivos CSV")

# Leer y concatenar
dfs = []
for archivo in archivos:
    print(f"📂 Leyendo {archivo}...")
    df = pd.read_csv(archivo)

     # Crear columna "fuente" con el nombre del archivo sin la extensión .csv
    df["fuente"] = os.path.splitext(os.path.basename(archivo))[0]
    dfs.append(df)

# Unir todo en un solo DataFrame
df_final = pd.concat(dfs, ignore_index=True)

# Eliminar duplicados (opcional)
#df_final.drop_duplicates(inplace=True)

# Guardar en un único CSV
df_final.to_csv("resultados_unidos.csv", index=False, encoding="utf-8-sig")

print("✅ Archivos unidos en resultados_unidos.csv")

🔎 Encontrados 7 archivos CSV
📂 Leyendo datos\Donde alojarse.csv...
📂 Leyendo datos\Donde comer.csv...
📂 Leyendo datos\Donde informarse.csv...
📂 Leyendo datos\Que Hacer.csv...
📂 Leyendo datos\Que visitar.csv...
📂 Leyendo datos\Servicios.csv...
📂 Leyendo datos\Turismo transporte.csv...
✅ Archivos unidos en resultados_unidos.csv


In [53]:
from selenium import webdriver  
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os

# === 1. Configuración Selenium ===
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 15)

# === 2. Leer CSV existente ===
csv_file = r"datos/Donde alojarse.csv"
df = pd.read_csv(csv_file, encoding="utf-8-sig")

# Añadimos columnas fijas si no existen
for col in ["coordenadas", "web"]:
    if col not in df.columns:
        df[col] = None

# === 3. Comprobar progreso previo ===
progreso_file = "progreso.txt"
if os.path.exists(progreso_file):
    with open(progreso_file, "r") as f:
        last_index = int(f.read().strip())
    print(f"♻️ Retomando desde la fila {last_index}")
else:
    last_index = 0

# === 4. Recorrer desde la última fila procesada ===
for i, row in df.iloc[last_index:].iterrows():
    url = row["link"]
    print(f"🌐 Procesando fila {i} -> {row['nombre']} -> {url}")

    try:
        driver.get(url)
        time.sleep(2)

        # Extraer coordenadas
        try:
            coords_elem = wait.until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='row padding-top-s']"))
            )
            coordenadas = coords_elem.text.replace("Coordenadas:", "").strip()
        except:
            coordenadas = None

        # Extraer web oficial
        try:
            web_elem = driver.find_element(By.XPATH, "//div[@class='media-body cont-info-contact']/a")
            web = web_elem.get_attribute("href")
        except:
            web = None

        # Buscar TODOS los bloques info-especial
        bloques_info = driver.find_elements(By.CLASS_NAME, "info-especial")

        for bloque in bloques_info:
            grupos = bloque.find_elements(By.CLASS_NAME, "bg-group") + \
                     bloque.find_elements(By.CLASS_NAME, "grupo.desplegador.fila")

            for g in grupos:
                texto = g.text.strip()
                if not texto:
                    continue

                lineas = texto.split("\n")
                titulo = lineas[0].strip()
                col_name = titulo.lower().replace(" ", "_")

                if col_name not in df.columns:
                    df[col_name] = None

                if "Número de cuartos" not in texto:
                    contenido = " | ".join(lineas[1:]).strip()
                    df.at[i, col_name] = contenido
                else:
                    filas = lineas[2:]
                    contenido_limpio = []
                    for fila in filas:
                        partes = fila.split()
                        if len(partes) < 3:
                            continue
                        habitacion = " ".join(partes[:-2])
                        cuartos = partes[-2]
                        plazas = partes[-1]
                        registro = f"Habitación: {habitacion} | Número de cuartos: {cuartos} | Número de plazas: {plazas}"
                        contenido_limpio.append(registro)
                    df.at[i, col_name] = "; ".join(contenido_limpio)

        # Guardar las columnas base
        df.at[i, "coordenadas"] = coordenadas
        df.at[i, "web"] = web

        # === Guardar progreso ===
        df.to_csv("Donde alojarse.csv", index=False, encoding="utf-8-sig")
        with open(progreso_file, "w") as f:
            f.write(str(i + 1))  # +1 porque ya procesamos esta fila

    except Exception as e:
        print(f"⚠️ Error en fila {i} -> {url}: {e}")
        # Guardar progreso aún si hay error
        with open(progreso_file, "w") as f:
            f.write(str(i))
        continue

driver.quit()
print("✅ Proceso completado. CSV actualizado y progreso guardado.")


🌐 Procesando fila 0 -> CASA CRUCEIRO -> https://www.turismo.gal/recurso/-/detalle/2104/-casa-cruceiro?langId=es_ES&tp=71&ctre=134
🌐 Procesando fila 1 -> 2 CASAS -> https://www.turismo.gal/recurso/-/detalle/210625000582/2-casas?langId=es_ES&tp=68&ctre=130
🌐 Procesando fila 2 -> 9 OLAS -> https://www.turismo.gal/recurso/-/detalle/240801000039/9-olas?langId=es_ES&tp=67&ctre=135
🌐 Procesando fila 3 -> A AVOA MARIA -> https://www.turismo.gal/recurso/-/detalle/35940/a-avoa-maria-?langId=es_ES&tp=71&ctre=134
🌐 Procesando fila 4 -> A BOA ESTRELA -> https://www.turismo.gal/recurso/-/detalle/210610000268/a-boa-estrela?langId=es_ES&tp=67&ctre=135
🌐 Procesando fila 5 -> A BOUZA -> https://www.turismo.gal/recurso/-/detalle/15736/a-bouza?langId=es_ES&tp=71&ctre=134
🌐 Procesando fila 6 -> A CAMBOA -> https://www.turismo.gal/recurso/-/detalle/11389/a-camboa?langId=es_ES&tp=71&ctre=134
🌐 Procesando fila 7 -> A CAMELIA -> https://www.turismo.gal/recurso/-/detalle/28859/a-camelia?langId=es_ES&tp=67&ctre=

KeyboardInterrupt: 